# Importa bibliotecas

In [1]:
import os
path = os.path.abspath('')
# print(path)
items = os.listdir(path)
if 'Funcoes_New.py' not in items:
    ! jupyter nbconvert --to 'python' Funcoes_New.ipynb
import Funcoes_New as func

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import ipywidgets as widgets
from bqplot import pyplot as bqplt
import bqplot as bq
%matplotlib notebook 

# Carrega banco de dados

In [2]:
equips_data = func.get_all_equips_data()
OS_data = func.get_all_OS_data()
OS_data = OS_data.dropna(subset=['Tipo Equip.'])
material_cost_data = func.load_material_cost_data()
external_cost_data = func.load_external_cost_data()
# equip_type = np.sort(equips_data['Tipo Equipamento'].unique())
equip_type = np.sort(equips_data['Tipo Equipamento'].unique())

# Gera interface

In [3]:
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

In [4]:
# from bqplot.interacts import PanZoom
# from traitlets import link
# from collections import OrderedDict
# buttonWidth = '50px'



def update_data(equip,start_date,end_date):
    import pandas as pd
    import numpy as np
    from datetime import date
    today = pd.to_datetime(date.today()).to_numpy()
    if len(equip)==1:
        equip = equip[0]
    available_equips, equip_amount_data = func.get_available_equip(equip,equips_data,OS_data)
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    x_data1,y_data1,x_data2,y_data2 = func.arrange_equip_data_to_plot(available_equips,equip_amount_data,start_date,end_date)
    
    #Checks for empty data (necessary to adjust y axis limits)
    empty_flag1,between_two_dates1 = func.check_for_empty_data(equip_amount_data,start_date,end_date)
    empty_flag2,between_two_dates2 = func.check_for_empty_data(available_equips,start_date,end_date)
    if empty_flag2==False:
        ymin = np.amin(available_equips[between_two_dates2]['Quantidade Disponível'].values.astype(int))
    else:
        ymin = 0
    if empty_flag1==False:
        ymax = np.amax(equip_amount_data[between_two_dates1]['Quantidade de Equipamentos'].values.astype(int))
    else:
        ymax = np.amax(equip_amount_data['Quantidade de Equipamentos'].values.astype(int))
#     #adds last point to masks
#     if end_date>=today:
#         between_two_dates1 = np.append(between_two_dates1,True)
#         between_two_dates2 = np.append(between_two_dates2,True)
#     else:
#         between_two_dates1 = np.append(between_two_dates1,False)
#         between_two_dates2 = np.append(between_two_dates2,False)
#     #adds first point to masks
#     if between_two_dates1[0]==True:
#         between_two_dates1 = np.insert(between_two_dates1,0,True)
#     else:
#         between_two_dates1 = np.insert(between_two_dates1,0,False)
#     if between_two_dates2[0]==True:
#         between_two_dates2 = np.insert(between_two_dates2,0,True)
#     else:
#         between_two_dates2 = np.insert(between_two_dates2,0,False)
    
        
    
    return(x_data1,y_data1,x_data2,y_data2,ymin,ymax)

def update_data2(equip,start_date,end_date):
    import pandas as pd
    if len(equip)==1:
        equip = equip[0]
    #TEMP###
    available_equips, equip_amount_data = func.get_available_equip(equip,equips_data,OS_data)
    ######
    material_monthly_cost = func.get_equip_monthly_cost(equip,material_cost_data)
    external_monthly_cost = func.get_equip_monthly_cost(equip,external_cost_data)
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    x_data1,y_data1,x_data2,y_data2,x_data3,y_data3 = func.arrange_cost_data_to_plot(material_monthly_cost,external_monthly_cost,equip_amount_data,start_date,end_date)
#     print(x_data2[-1])
#     print(x_data1[-1])
    #Checks for empty data (necessary to adjust y axis limits)
    empty_flag1,between_two_dates1 = func.check_for_empty_data(equip_amount_data,start_date,end_date)
    empty_flag2,between_two_dates2 = func.check_for_empty_data(material_monthly_cost,start_date,end_date)
    empty_flag3,between_two_dates3 = func.check_for_empty_data(external_monthly_cost,start_date,end_date)
    
    if empty_flag1==False:
        ymin = np.amin(equip_amount_data[between_two_dates1]['Quantidade de Equipamentos'].values.astype(int))
        ymax = np.amax(equip_amount_data[between_two_dates1]['Quantidade de Equipamentos'].values.astype(int))
    else:
        ymin = 0
        ymax = np.amax(equip_amount_data['Quantidade de Equipamentos'].values.astype(int))
    
    if selected_analysis.value==1:
        if empty_flag2==False:
            ymax2 = np.amax(material_monthly_cost[between_two_dates2].values)
        else:
            ymax2 = 1
    elif selected_analysis.value==2:
        if empty_flag3==False:
            ymax2 = np.amax(external_monthly_cost[between_two_dates3].values)
        else:
            ymax2 = 1
    else:
        if empty_flag2==False:
            if empty_flag3==False:
                # matches lenghts of x_data2 with x_data3 (add missing dates in x_data and add zeros in missing y_data) 
                i=0
                while x_data3[i] < x_data2[i]:
                    x_data2 = np.insert(x_data2,i,x_data3[i])
                    y_data2 = np.insert(y_data2,0,0)
                    i+=1
                i=0
                while x_data2[i] < x_data3[i]:
                    x_data3 = np.insert(x_data3,i,x_data2[i])
                    y_data3 = np.insert(y_data3,0,0)
                    i+=1
                if len(x_data2)>len(x_data3):
                    y_data3 = np.append(y_data3,np.zeros(len(x_data2)-len(x_data3)))
                    x_data3 = np.append(x_data3,x_data2[len(x_data3):])
                    x_data3 = pd.to_datetime(x_data3)
                elif len(x_data2)<len(x_data3):
                    y_data2 = np.append(y_data2,np.zeros(len(x_data3)-len(x_data2)))
                    x_data2 = np.append(x_data2,x_data3[len(x_data2):])
                    x_data2 = pd.to_datetime(x_data2)
                
                ymax2 = np.amax(np.concatenate((material_monthly_cost[between_two_dates2].values,
                                               external_monthly_cost[between_two_dates3].values)))
            else:
                ymax2 = np.amax(material_monthly_cost[between_two_dates2].values)
        else:
            if empty_flag3==False:
                 ymax2 = np.amax(external_monthly_cost[between_two_dates3].values)
            else:
                ymax2 = 1
    
    
    return(x_data1,y_data1,x_data2,y_data2,x_data3,y_data3,ymin,ymax,ymax2)

title = widgets.Label(value="Interface de Análise Temporal do GETS",
                      layout=widgets.Layout(border= '2px solid black',width='100%', display='flex', align_items='center',
                                           justify_content='center'))

tab_titles = ['Quantidade','Custo']
tab = widgets.Tab()
[tab.set_title(i, title) for i, title in enumerate(tab_titles)]

fig = bq.Figure()
tb = bq.Toolbar(figure=fig)
fig2 = bq.Figure()
tb2 = bq.Toolbar(figure=fig2)

#########################
# pz = PanZoom(scales={'x': [x_dt], 'y': [y_sc]})
# pzx = PanZoom(scales={'x': [x_dt]})
# pzy = PanZoom(scales={'y': [y_sc], })

# #
# zoom_interacts = widgets.ToggleButtons(
#                                         options=OrderedDict([
#                                             ('xy ', pz), 
#                                             ('x ', pzx), 
#                                             ('y ', pzy),   
#                                             (' ', None)]),
#                                             icons = ["arrows", "arrows-h", "arrows-v", "stop"],
#                                             tooltips = ["zoom/pan in x & y", "zoom/pan in x only", "zoom/pan in y only", "cancel zoom/pan"]
#                                         )
# zoom_interacts.style.button_width = buttonWidth

# ResetZoomButton = widgets.Button(
#     description='',
#     disabled=False,
#     button_style='', # 'success', 'info', 'warning', 'danger' or ''
#     tooltip='Reset zoom',
#     icon='arrows-alt'
# )

# def resetZoom(new):
#     # Reset the x and y axes on the figure
#     fig.axes[0].scale.min = None
#     fig.axes[1].scale.min = None
#     fig.axes[0].scale.max = None
#     fig.axes[1].scale.max = None  
    
# ResetZoomButton.on_click(resetZoom)
# ResetZoomButton.layout.width = buttonWidth

# link((zoom_interacts, 'value'), (fig, 'interaction'))
# tb = widgets.HBox([zoom_interacts,ResetZoomButton])
########################
label1 = widgets.Label(value="Tipo de Medida:")
label2 = widgets.Label(value="Equipamento:")
label3 = widgets.Label(value="Data Inicial:")
label4 = widgets.Label(value="Data Final:")
label5 = widgets.Label(value="Mostrar Tendência?")
error_label = widgets.Label(value='')

analysis_types = [('Custo Total de Material/Mês',1), ('Custo Total de Serviço Externo/Mês',2),('Custos Totais/Mês',3)]
selected_analysis = widgets.Dropdown(
    options=analysis_types,
    value=1,
    description='',
    disabled=False,
)



selected_equips = widgets.Combobox(
    options=equip_type.tolist(),
    placeholder='Ex: VENTILADOR UTI',
    value='',
    #rows=10,
    ensure_option=True,
    disabled=False
)
tendency = widgets.Checkbox(
    value=False,
    description='Mostrar linha de tendência',
    disabled=False,
    indent=False
)
# hbox = widgets.HBox([label5, tendency],layout=widgets.Layout(display='flex',align_items='stretch'))
selected_start_date = widgets.DatePicker(
    description='',
    disabled=False,
    value=pd.to_datetime('2013-01-01')
)
selected_end_date = widgets.DatePicker(
    description='',
    disabled=False,
    value=pd.to_datetime('2020-09-01')
)


def on_change(b):
    error_label.value = ''
    if selected_start_date.value==None:
        error_label.value = 'Por favor selecione uma data inicial!'
    elif selected_end_date.value==None:
        error_label.value = 'Por favor selecione uma data final!'
    elif selected_equips.value=='':
        error_label.value = 'Por favor selecione um equipamento!'
    else:
        
        if tab.selected_index==0:
            
            
            x_data1,y_data1,x_data2,y_data2,ymin,ymax = update_data(selected_equips.value,selected_start_date.value,selected_end_date.value)
            
            
            x_dt = bq.DateScale()
            y_sc = bq.LinearScale()
            interp = 'linear'
            def_tt = bq.Tooltip(fields=['x', 'y'], formats=['%d-%b-%y', 'd'],labels=['Data','Quant.'])

            scatter1 = bq.Scatter(x=x_data1, y=y_data1, colors=['#ff77ff00'], stroke_width=1, display_legend=False,
                                  scales={'x': x_dt, 'y': y_sc}, tooltip=def_tt)
            line1 = bq.Lines(x=x_data1, y=y_data1, scales={'x': x_dt, 'y': y_sc}, display_legend=True, labels=['Ativo'],
                             interpolation=interp)
            line1.fill = 'bottom'
            line1.marker = 'square'
            line1.marker_size = 20
            line1.fill_opacities = [0.2]
            scatter2 = bq.Scatter(x=x_data2, y=y_data2, colors=['#ff77ff00'], stroke_width=1, display_legend=False,
                                  scales={'x': x_dt, 'y': y_sc}, tooltip=def_tt)
            line2 = bq.Lines(x=x_data2, y=y_data2, scales={'x': x_dt, 'y': y_sc}, display_legend=True, 
                             labels=['Disponível'],interpolation=interp, tooltip=def_tt)
            line2.fill = 'bottom'
            line2.marker = 'circle'
            line2.fill_opacities = [0.2]
            line2.colors = ['DarkOrange']
            line2.marker_size = 8
            line2.opacities = [.5]

            ax_x = bq.Axis(scale=x_dt, label='Data', 
                          tick_style={'stroke': 'black', 'font-size': 12}, tick_format="%b %Y", num_ticks=8)
            x_dt.min = pd.to_datetime(selected_start_date.value)
            x_dt.max = pd.to_datetime(selected_end_date.value)
#             ax_x.tick_rotate=-45
            if (pd.to_datetime(selected_end_date.value) - pd.to_datetime(selected_start_date.value)).days<240:
                ax_x.tick_format = "%d-%b-%y"
            
#             print('ymin,ymax=',ymin, ymax)
            ax_y = bq.Axis(scale=y_sc, orientation='vertical', label='Quantidade de Equipamentos', 
                           label_style={'stroke': 'black', 'font-size': 12},
                           tick_style={'stroke': 'black', 'font-size': 12},grid_lines='none')
            y_sc.min = np.around(0.9*ymin)
            y_sc.max = np.around(1.1*ymax)
            
#             start = np.around(0.9*ymin).astype(int)
#             stop = np.around(1.1*ymax).astype(int)
#             max_range = stop - start
#             npoints = 20
#             if max_range < npoints:
#                 y_sc.domain = list(range(start,stop))
#             else:
#                 step = max_range//npoints
#                 y_sc.domain = list(range(start,stop,step))

            fig.marks = [line1,line2,scatter1,scatter2]
            fig.axes = [ax_x,ax_y]
            fig.legend_location = 'top-left'
            
        else:
            x_data1,y_data1,x_data2,y_data2,x_data3,y_data3,ymin,ymax,ymax2 = update_data2(selected_equips.value,
                                                                                           selected_start_date.value,
                                                                                           selected_end_date.value)
            x_dt = bq.DateScale()
            y_sc1 = bq.LinearScale()
            y_sc2 = bq.LinearScale()
            interp = 'linear'
            def_tt = bq.Tooltip(fields=['x', 'y'], formats=['%d-%b-%y', 'd'],labels=['Data','Quant.'])
#             out = ipw.Output()
#     def hover_handler(m, hover_event):
#         out.clear_output()
#         with out:
#             print(hover_event)
            scatter1 = bq.Scatter(x=x_data1, y=y_data1, colors=['#ff77ff00'], stroke_width=1, display_legend=False,
                                  scales={'x': x_dt, 'y': y_sc1}, tooltip=def_tt)
#             scatter.on_hover(hover_handler)
            line1 = bq.Lines(x=x_data1, y=y_data1, scales={'x': x_dt, 'y': y_sc1}, display_legend=True, 
                             labels=['Ativo'],interpolation=interp)
            line1.fill = 'bottom'
            line1.marker = 'square'
            line1.marker_size = 20
            line1.fill_opacities = [0.2]
            def_tt2 = bq.Tooltip(fields=['x', 'y'], formats=['%b %Y', '.2f'],labels=['Mês/Ano','Custo'])
            bar2 = bq.Bars(x=x_data2,y=y_data2, scales={'x': x_dt, 'y': y_sc2},tooltip=def_tt2,display_legend=True, 
                             labels=['Materiais'])
            bar2.colors = ['Red']
            bar2.opacities = [0.5]
            
            bar3 = bq.Bars(x=x_data3,y=y_data3, scales={'x': x_dt, 'y': y_sc2},tooltip=def_tt2,display_legend=True, 
                             labels=['Serviços Externos'])
            bar3.colors = ['Orange']
            bar3.opacities = [0.5]
            
            ax_x = bq.Axis(scale=x_dt, label='Data', xlim=(selected_start_date.value,selected_end_date.value),
                          tick_style={'stroke': 'black', 'font-size': 12}, tick_format="%b %Y", num_ticks=8)
            x_dt.min = pd.to_datetime(selected_start_date.value)
            x_dt.max = pd.to_datetime(selected_end_date.value)
#             ax_x.tick_rotate=-45
            if (pd.to_datetime(selected_end_date.value) - pd.to_datetime(selected_start_date.value)).days<240:
                ax_x.tick_format = "%d-%b-%y"
            
            ax_y1 = bq.Axis(scale=y_sc1, orientation='vertical', label='Quantidade de Equipamentos',
                            tick_style={'stroke': 'black', 'font-size': 12},grid_lines='none')
            ax_y2 = bq.Axis(scale=y_sc2, orientation='vertical', label=selected_analysis.label, 
                            side='right', label_color='OrangeRed',color='red',label_offset='3.5em',
                           tick_style={'stroke': 'OrangeRed', 'font-size': 12},grid_lines='dashed',grid_color='OrangeRed')
            y_sc1.min = np.around(0.9*ymin)
            y_sc1.max = np.around(1.1*ymax)
            y_sc2.min = 0
            y_sc2.max = 1.1*ymax2
            if selected_analysis.value==1:
                fig2.marks = [line1,bar2,scatter1]
            elif selected_analysis.value==2:
                fig2.marks = [line1,bar3,scatter1]
            else:
                bar = bq.Bars(x=x_data2,y=[y_data2,y_data3], scales={'x': x_dt, 'y': y_sc2}, type='grouped',
                              colors=['Red','Orange'],tooltip=def_tt2,display_legend=True, 
                             labels=['Materiais','Serviços Externos']) 
                bar.opacities = [0.5, 0.5]
                fig2.marks = [line1,bar,scatter1]

            bar2.interactions = {
                'legend_hover': 'highlight_axes',
                'hover': 'tooltip', 
            }
            
            fig2.axes = [ax_x,ax_y1,ax_y2]
            fig2.legend_location = 'top-left'

            
# def on_toolbar(b):
#     print(tb._panzoom)
#     print(b)
#     print('bla')
            
selected_analysis.observe(on_change,names='value')
selected_equips.observe(on_change,names='value')
selected_start_date.observe(on_change,names='value')
selected_end_date.observe(on_change,names='value')
tab.observe(on_change,names='selected_index')
# tb.observe(on_toolbar,'_panning')

vbox = widgets.VBox([label2,
              selected_equips,
              label3,
              selected_start_date,
              label4,
              selected_end_date,
              tendency,
#               button_plot,
              error_label
             ])

vbox2 = widgets.VBox([label1,
              selected_analysis,
              label2,
              selected_equips,
              label3,
              selected_start_date,
              label4,
              selected_end_date,
#               button_plot,
              error_label
             ])

# center_layout = widgets.Layout(display='flex-grow',
#                 align_items='center')
vbox_fig = widgets.VBox([fig,tb], align_self='stretch')#,layout=center_layout)
vbox_fig2 = widgets.VBox([fig2,tb2], align_self='stretch')#,layout=center_layout)

app = widgets.AppLayout(header=None,
          left_sidebar=vbox,
          center=vbox_fig,
          right_sidebar=None,
          footer=None,
          pane_widths=[2, 4, 0],
          pane_heights=[0,9,0],
          layout=widgets.Layout(border= ''))

app2 = widgets.AppLayout(header=None,
          left_sidebar=vbox2,
          center=vbox_fig2,
          right_sidebar=None,
          footer=None,
          pane_widths=[2, 4, 0],
          pane_heights=[0,9,0],
          layout=widgets.Layout(border= ''))

children = [app,app2]
tab.children = children

header = widgets.HBox()
# header_layout = Layout(width='100px',height='100px')
# header_layout.width = '100%'
# ceb_logo.object_fit = 'contain'


header.children = [title]

# vb = VBox()
# h = HTML(value='<{size}>Examples of <code>object_fit</code> with large image</{size}>'.format(size=caption_size))
# vb.layout.align_items = 'center'
# hb = HBox()
# hb.layout = hbox_layout
# hb.children = boxes

# vb.children = [h, hb]
# vb




# header = widgets.HBox([title,ceb_logo])
# header.width = '100%'
# header.layout.align_items='center'
# header.layout.justify_itens = 'center'
main_app = widgets.VBox([header,tab])
main_app

In [5]:
print('yay')

yay
